# Reusable Codes in Python

<b style="font-size:20px"> <a href="#Create py file and store variable">Create py file and store variable</a><br>
<b style="font-size:20px"> <a href="#Import IPython Notebooks as Module">Import IPython Notebooks as Module</a><br>
<b style="font-size:20px"> <a href="#Calculate execution time">Calculate execution time</a><br>
<b style="font-size:20px"> <a href="#Running Shell Commands">Running Shell Commands</a><br>
<b style="font-size:20px"> <a href="#Crawl over a folder">Crawl over a folder</a>

<b style="font-size:15px"><a id="Create py file and store variable">Create py file and store variable</a>

In [2]:
"""
Create py file to store variable
To include save_var.py in other code use "from save_var import *"
If you want to save a class variable (e.g. bigram_pos_tagger.transitions) (bigram_pos_tagger is a class object)
    Assign it to a local variable (e.g. transitions = bigram_pos_tagger.transitions)
    Then, call save_var(transitions)
http://stackoverflow.com/questions/18425225/getting-the-name-of-a-variable-as-a-string
"""
#f = open( 'save_var.py', 'w' ) # Put this in the beginning of your program. It will create a new file
#f.close()

def save_var(var):
    import inspect
    from collections import defaultdict

    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    var_name = [var_name for var_name, var_val in callers_local_vars if var_val is var][0] # fetch variable name

    save_var_file = open('save_var.py', 'a')  # append to the file created
    save_var_file.write('#' + repr(type(var)) + '\n')
    save_var_file.write('#Len: ' + repr(len(var)) + '\n')

    # to print defaultdict variable without its type
    if isinstance(var, defaultdict):
        save_var_file.write(var_name + ' = ' + repr(dict(var)) + '\n\n')
    else:
        save_var_file.write(var_name + ' = ' + repr(var) + '\n\n')

    save_var_file.close()
    
"""
Create py file to store variable: Works for GLOBAL variables only
To include save_var.py in other code use "from save_var import *"
If you want to save a class variable (e.g. bigram_pos_tagger.transitions) (bigram_pos_tagger is a class object)
    Assign it to a local variable (e.g. transitions = bigram_pos_tagger.transitions)
    Then, call save_var(transitions)
http://stackoverflow.com/questions/1534504/convert-variable-name-to-string
"""

#f = open( 'save_var.py', 'w' ) # Put this in the beginning of your program. It will create a new file
#f.close()

# call save_var and pass the variable you want to save
def save_var(var):
    var_name = [name for name in globals() if globals()[name] is var][0] # fetch variable name
        
    f = open( 'save_var.py', 'a' ) # append to the file created
    f.write('#' + repr(type(var)) + '\n' )
    f.write('#Len: ' + repr(len(var)) + '\n' )
    
    # to print defaultdict variable without its type
    if isinstance(var, defaultdict):
        f.write(var_name + ' = ' + repr(dict(var)) + '\n\n')
    else:
        f.write(var_name + ' = ' + repr(var) + '\n\n')
    
    f.close()

<b style="font-size:15px"><a id="Import IPython Notebooks as Module">Import IPython Notebooks as Module</a>

In [1]:
"""
Import IPython Notebooks as Module.
http://jupyter-notebook.readthedocs.io/en/latest/examples/Notebook/Importing%20Notebooks.html
http://nbviewer.jupyter.org/gist/minrk/6011986
"""

import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
        
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

# Module to import
#from bigram_hmm import BigramHMM

<b style="font-size:15px"><a id="Calculate execution time">Calculate execution time</a>

In [1]:
import time

"""
https://www.w3resource.com/python-exercises/python-basic-exercise-65.php
"""

def time_taken(start, end):
    """Human readable time between `start` and `end`

    :param start: time.time()
    :param end: time.time()
    :returns: day:hour:minute:second.millisecond
    """
    my_time = end-start
    day = my_time // (24 * 3600)
    my_time = my_time % (24 * 3600)
    hour = my_time // 3600
    my_time %= 3600
    minutes = my_time // 60
    my_time %= 60
    seconds = my_time
    milliseconds = ((end - start)-int(end - start))
    day_hour_min_sec = str('%02d' % int(day))+":"+str('%02d' % int(hour))+":"+str('%02d' % int(minutes))+":"+str('%02d' % int(seconds)+"."+str('%.3f' % milliseconds)[2:])
    return day_hour_min_sec


In [2]:
start_time = time.time()

In [3]:
end_time = time.time()

In [5]:
print("day, hour, minute, second.millisecond->", time_taken(start_time, end_time))

day, hour, minute, second.millisecond-> 00:00:00:03.007


<b style="font-size:15px"><a id="Running Shell Commands">Running Shell Commands</a>

In [1]:
import subprocess, sys

def execute(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

    # Poll process for new output until finished
    while True:
        nextline = process.stdout.readline()
        if nextline == '' and process.poll() is not None:
            break
        sys.stdout.write(nextline)
        sys.stdout.flush()

    output = process.communicate()[0]
    exitCode = process.returncode

    if (exitCode == 0):
        return output
    else:
        raise ProcessException(command, exitCode, output)


command = "dir"

print(command)
execute(command)

dir
 Volume in drive C is Windows8_OS
 Volume Serial Number is AADC-2C7F

 Directory of C:\Users\GyanT\Documents\GitHub\Python-Programming

18/Jun/18  10:45 AM    <DIR>          .
18/Jun/18  10:45 AM    <DIR>          ..
11/Jun/18  09:52 AM    <DIR>          .ipynb_checkpoints
11/Jun/18  09:52 AM             5,872 CodeChallenge.ipynb
11/Jun/18  09:52 AM             1,946 FileReadWrite.ipynb
11/Jun/18  09:52 AM               182 FileReadWrite.txt
11/Jun/18  09:52 AM    <DIR>          files
11/Jun/18  09:52 AM             1,789 Hair.ipynb
11/Jun/18  09:52 AM            12,106 ImagesRename.ipynb
11/Jun/18  09:52 AM            16,338 Introduction.ipynb
11/Jun/18  09:52 AM    <DIR>          Matplotlib API
11/Jun/18  09:52 AM    <DIR>          NLTK
11/Jun/18  09:52 AM    <DIR>          NumPy
11/Jun/18  09:52 AM            11,378 Problems.ipynb
11/Jun/18  09:52 AM    <DIR>          Python Standard Library
11/Jun/18  09:52 AM    <DIR>          PyTorch
18/Jun/18  10:45 AM            11,218 Reus

''

<b style="font-size:15px"><a id="Crawl over a folder">Crawl over a folder</a>

In [2]:
import os

for root, dirs, filenames in os.walk(os.getcwd()):
    for a_file in filenames:
        if a_file.endswith(".ipynb"):
            print(a_file)

CodeChallenge.ipynb
FileReadWrite.ipynb
Hair.ipynb
ImagesRename.ipynb
Introduction.ipynb
Problems.ipynb
ReusableCode.ipynb
CodeChallenge-checkpoint.ipynb
FileReadWrite-checkpoint.ipynb
Hair-checkpoint.ipynb
ImagesRename-checkpoint.ipynb
Introduction-checkpoint.ipynb
Problems-checkpoint.ipynb
ReusableCode-checkpoint.ipynb
GUI_toolkit.ipynb
image.ipynb
patches.ipynb
pyplot.ipynb
GUI_toolkit-checkpoint.ipynb
image-checkpoint.ipynb
patches-checkpoint.ipynb
pyplot-checkpoint.ipynb
nltk.ipynb
nltk-checkpoint.ipynb
numpy.ipynb
numpy-checkpoint.ipynb
csv.ipynb
math.ipynb
csv-checkpoint.ipynb
math-checkpoint.ipynb
ProcessAnkiWordData-checkpoint.ipynb
1.tensor_tutorial.ipynb
2.autograd_tutorial.ipynb
3.neural_networks_tutorial.ipynb
4.cifar10_tutorial_nc1_1.ipynb
4.cifar10_tutorial_nc1_1_short.ipynb
1.tensor_tutorial-checkpoint.ipynb
2.autograd_tutorial-checkpoint.ipynb
3.neural_networks_tutorial-checkpoint.ipynb
4.cifar10_tutorial - Copy-checkpoint.ipynb
4.cifar10_tutorial-checkpoint.ipynb
4.ci

In [2]:
"""
https://www.tutorialspoint.com/How-to-use-Glob-function-to-find-files-recursively-in-Python
Python 3
The glob module supports the "**" directive (which is parsed only if you pass recursive flag)
which tells python to look recursively in the directories
"""

import glob

for filename in glob.iglob('*/**/*.ipynb', recursive=True):
    print(filename)

Matplotlib API\GUI_toolkit.ipynb
Matplotlib API\image.ipynb
Matplotlib API\patches.ipynb
Matplotlib API\pyplot.ipynb
NLTK\nltk.ipynb
NumPy\numpy.ipynb
Python Standard Library\csv.ipynb
Python Standard Library\math.ipynb
PyTorch\1 Deep Learning with PyTorch - A 60 Minute Blitz\1.tensor_tutorial.ipynb
PyTorch\1 Deep Learning with PyTorch - A 60 Minute Blitz\2.autograd_tutorial.ipynb
PyTorch\1 Deep Learning with PyTorch - A 60 Minute Blitz\3.neural_networks_tutorial.ipynb
PyTorch\1 Deep Learning with PyTorch - A 60 Minute Blitz\4.cifar10_tutorial_nc1_1.ipynb
PyTorch\1 Deep Learning with PyTorch - A 60 Minute Blitz\4.cifar10_tutorial_nc1_1_short.ipynb
